In [1]:
!rm simpsons.sqlite
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, String, Integer, Float, Text

In [2]:
url = 'simpsons_script_lines.csv'
url_characters = 'simpsons_characters.csv'
url_locations = 'simpsons_locations.csv'
url_episodes = 'simpsons_episodes.csv'

In [3]:
df_script = pd.read_csv(url, low_memory=False)
df_characters = pd.read_csv(url_characters)
df_locations = pd.read_csv(url_locations)
df_episodes = pd.read_csv(url_episodes)

In [4]:
df_script.columns.tolist()

['id',
 'episode_id',
 'number',
 'raw_text',
 'timestamp_in_ms',
 'speaking_line',
 'character_id',
 'location_id',
 'raw_character_text',
 'raw_location_text',
 'spoken_words',
 'normalized_text',
 'word_count']

In [5]:
df_characters.columns.tolist()

['id', 'name', 'normalized_name', 'gender']

In [6]:
df_locations.columns.tolist()

['id', 'name', 'normalized_name']

In [7]:
df_episodes.columns.tolist()

['id',
 'title',
 'original_air_date',
 'production_code',
 'season',
 'number_in_season',
 'number_in_series',
 'us_viewers_in_millions',
 'views',
 'imdb_rating',
 'imdb_votes',
 'image_url',
 'video_url']

In [8]:
engine = create_engine("sqlite:///simpsons.sqlite")
Base = declarative_base(bind=engine)

In [9]:
class Scripts(Base):
    __tablename__ = 'scripts'    
    id = Column(Integer, primary_key=True)
    episode_id = Column(Text)
    number = Column(Text)
    raw_text = Column(Text)
    timestamp_in_ms = Column(Text)
    speaking_line = Column(Text)
    character_id = Column(Text)
    location_id = Column(Text)
    raw_character_text = Column(Text)
    raw_location_text = Column(Text)
    spoken_words = Column(Text)
    normalized_text = Column(Text)
    word_count = Column(Text)

class Characters(Base):
    __tablename__ = 'characters'
    id = Column(Integer, primary_key=True)
    name = Column(Text)
    normalized_name = Column(Text)
    gender = Column(Text)
    
class Locations(Base):
    __tablename__ = 'locations'
    id = Column(Integer, primary_key=True)
    name = Column(Text)
    normalized_name = Column(Text)
    
class Episodes(Base):
    __tablename__ = 'episodes'
    id = Column(Integer, primary_key=True)
    title = Column(Text)
    original_air_date = Column(Text)
    production_code = Column(Text)
    season = Column(Text)
    number_in_season = Column(Text)
    number_in_series = Column(Text)
    us_viewers_in_millions = Column(Text)
    views = Column(Text)
    imdb_rating = Column(Text)
    imdb_votes = Column(Text)
    image_url = Column(Text)
    video_url = Column(Text)

In [10]:
Base.metadata.create_all()

In [11]:
script_dict = df_script.to_dict(orient='records')
location_dict = df_locations.to_dict(orient='records')
character_dict = df_characters.to_dict(orient='records')
episode_dict = df_episodes.to_dict(orient='records')

In [12]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [13]:
script_table = sqlalchemy.Table('scripts', metadata, autoload=True)
engine.execute(script_table.delete())
engine.execute(script_table.insert(), script_dict)

location_table = sqlalchemy.Table('locations', metadata, autoload=True)
engine.execute(location_table.delete())
engine.execute(location_table.insert(), location_dict)

character_table = sqlalchemy.Table('characters', metadata, autoload=True)
engine.execute(character_table.delete())
engine.execute(character_table.insert(), character_dict)

episode_table = sqlalchemy.Table('episodes', metadata, autoload=True)
engine.execute(episode_table.delete())
engine.execute(episode_table.insert(), episode_dict)

In [14]:
engine.execute("SELECT * FROM scripts LIMIT 5").fetchall()

[(1, '1', '0', '(Street: ext. street - establishing - night)', '8000', 'FALSE', None, '1.0', None, 'Street', None, None, None),
 (2, '1', '1', '(Car: int. car - night)', '8000', 'FALSE', None, '2.0', None, 'Car', None, None, None),
 (3, '1', '2', 'Marge Simpson: Ooo, careful, Homer.', '8000', 'TRUE', '1', '2.0', 'Marge Simpson', 'Car', 'Ooo, careful, Homer.', 'ooo careful homer', '3'),
 (4, '1', '3', "Homer Simpson: There's no time to be careful.", '10000', 'TRUE', '2', '2.0', 'Homer Simpson', 'Car', "There's no time to be careful.", 'theres no time to be careful', '6'),
 (5, '1', '4', "Homer Simpson: We're late.", '10000', 'TRUE', '2', '2.0', 'Homer Simpson', 'Car', "We're late.", 'were late', '2')]

In [15]:
engine.execute("SELECT * FROM locations LIMIT 5").fetchall()

[(1, 'Street', 'street'),
 (2, 'Car', 'car'),
 (3, 'Springfield Elementary School', 'springfield elementary school'),
 (4, 'Auditorium', 'auditorium'),
 (5, 'Simpson Home', 'simpson home')]

In [16]:
engine.execute("SELECT * FROM characters LIMIT 5").fetchall()

[(1, 'Marge Simpson', 'marge simpson', 'f'),
 (2, 'Homer Simpson', 'homer simpson', 'm'),
 (3, 'Seymour Skinner', 'seymour skinner', 'm'),
 (4, 'JANEY', 'janey', 'f'),
 (5, 'Todd Flanders', 'todd flanders', 'm')]

In [17]:
engine.execute("SELECT * FROM episodes LIMIT 5").fetchall()

[(1, 'Simpsons Roasting on an Open Fire', '1989-12-17', '7G08', '1', '1', '1', '26.7', '171408.0', '8.2', '3734.0', 'http://static-media.fxx.com/img/FX_Networks_-_FXX/617/479/Simpsons_01_08.jpg', 'http://www.simpsonsworld.com/video/273376835817'),
 (2, 'Bart the Genius', '1990-01-14', '7G02', '1', '2', '2', '24.5', '91423.0', '7.8', '1973.0', 'http://static-media.fxx.com/img/FX_Networks_-_FXX/265/167/Simpsons_01_02.jpg', 'http://www.simpsonsworld.com/video/283744835990'),
 (3, "Homer's Odyssey", '1990-01-21', '7G03', '1', '3', '3', '27.5', '78072.0', '7.5', '1709.0', 'http://static-media.fxx.com/img/FX_Networks_-_FXX/621/883/Simpsons_01_03.jpg', 'http://www.simpsonsworld.com/video/273381443699'),
 (4, "There's No Disgrace Like Home", '1990-01-28', '7G04', '1', '4', '4', '20.2', '67378.0', '7.8', '1701.0', 'http://static-media.fxx.com/img/FX_Networks_-_FXX/632/119/Simpsons_01_04__343617.jpg', 'http://www.simpsonsworld.com/video/273392195780'),
 (5, 'Bart the General', '1990-02-04', '7G0